# Dense Retrieval

## Setup

필요한 API 키, 그리고 관련 파이썬 라이브러리를 불러옵니다.

In [ ]:
# !pip install cohere 
# !pip install weaviate-client Annoy

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [ ]:
import cohere
co = cohere.Client(os.environ['COHERE_API_KEY'])

In [ ]:
import weaviate # 벡터 데이터베이스 라이브러리입니다.
auth_config = weaviate.auth.AuthApiKey(
    api_key=os.environ['WEAVIATE_API_KEY'])

In [ ]:
client = weaviate.Client(
    url=os.environ['WEAVIATE_API_URL'],
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.environ['COHERE_API_KEY'],
    }
)
client.is_ready() #check if True

## Part 1: 의미(semantic) 검색을 위한 벡터 데이터베이스

In [ ]:
def dense_retrieval(query, 
                    results_lang='en', 
                    properties = ["text", "title", "url", "views", "lang", "_additional {distance}"],
                    num_results=5):

    # 유사한 텍스트를 찾기 위한 세팅
    nearText = {"concepts": [query]}
    
    # 언어로 필터링(여기서는 영어)
    where_filter = {
    "path": ["lang"],
    "operator": "Equal",
    "valueString": results_lang
    }
    response = (
        client.query
        .get("Articles", properties)
        .with_near_text(nearText) # 추가된 내용
        .with_where(where_filter)
        .with_limit(num_results)
        .do()
    )

    result = response['data']['Get']['Articles']

    return result

In [ ]:
from utils import print_result

### 쉬운 질문

In [ ]:
query = "Who wrote Hamlet?"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

### 중간 질문

'dense retrieval'과 '키워드 검색'의 차이를 비교해보세요.

In [ ]:
query = "What is the capital of Canada?"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

In [ ]:
from utils import keyword_search

query = "What is the capital of Canada?"
keyword_search_results = keyword_search(query, client)
print_result(keyword_search_results)

### 복잡한 질문

In [ ]:
from utils import keyword_search

query = "Tallest person in history?"
keyword_search_results = keyword_search(query, client)
print_result(keyword_search_results)

In [ ]:
query = "Tallest person in history"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

심지어 다른 언어로 query를 전달해도 검색이 잘 되는 것을 확인할 수 있습니다.

In [ ]:
query = "أطول رجل في التاريخ"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

In [ ]:
query = "film about a time travel paradox"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

## Part 2: Semantic Search 구현

### 텍스트 아카이브 가져오기:

In [ ]:
from annoy import AnnoyIndex # 벡터 저장 라이브러리, 텍스트는 저장하지 못함
import numpy as np
import pandas as pd
import re

In [ ]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

### Chunking(덩어리 단위로 쪼개기): 

In [ ]:
# 문장 단위로 잘라 리스트를 만듭니다.
texts = text.split('.')

# 띄어쓰기와 줄 바꿈 문자가 결합된 것을 삭제합니다.
texts = np.array([t.strip(' \n') for t in texts])

In [ ]:
texts

문장 단위로 쪼개면 문맥이 없어서 정확한 의미를 전달하지 못할 가능성이 높습니다.  
그래서 아래와 같이 문단 단위로 쪼개기도 합니다.

In [ ]:
# 문단 단위로 쪼갭니다.
texts = text.split('\n\n')

# 띄어쓰기와 줄 바꿈 문자가 결합된 것을 삭제합니다.
texts = np.array([t.strip(' \n') for t in texts])

In [ ]:
texts

In [ ]:
# 문장 단위로 잘라 리스트를 만듭니다.
texts = text.split('.')

# 띄어쓰기와 줄 바꿈 문자가 결합된 것을 삭제합니다.
texts = np.array([t.strip(' \n') for t in texts])

위키피디아 사이트의 큰 장점 중 하나는 텍스트에 제목이 있다는 것입니다.  
이를 통해 같은 문장이라도 구체적인 의미를 담을 수 있도록, 각 문장의 앞에 문단의 제목을 붙여줍니다.

(사실 이런 방식은 굉장히 heuristic하긴 하지만 굉장히 직관적입니다.)

In [ ]:
title = 'Interstellar (film)'

texts = np.array([f"{title} {t}" for t in texts])

In [ ]:
texts

### 임베딩 구하기:

In [ ]:
response = co.embed(
    texts=texts.tolist() # 텍스트를 리스트로 변환합니다
).embeddings

In [ ]:
embeds = np.array(response)
embeds.shape # (문장 개수, 4096)

### 검색 인덱스 생성:

In [ ]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# 모든 벡ㅌ터에 검색 인덱스를 붙입니다.
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann') # 유사도 검색 인덱스를 저장합니다.

In [ ]:
pd.set_option('display.max_colwidth', None)

def search(query):

  # query의 임베딩을 구합니다.
  query_embed = co.embed(texts=[query]).embeddings

  # 가장 인접한 것을 탐색합니다.
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    3,
                                                  include_distances=True)
  # 결과를 데이터프레임으로 구성합니다.
  results = pd.DataFrame(data={'texts': texts[similar_item_ids[0]],
                              'distance': similar_item_ids[1]})

  print(texts[similar_item_ids[0]])
    
  return results

In [ ]:
query = "How much did the film make?"
search(query)

강의 마지막에, Annoy(유사한 것 중엔 FAISS)와 이전에 사용했던 Weaviate 등을 비교하는 내용이 나옵니다.  
전자는 세팅하기 쉽지만 벡터만 저장 가능하다는 단점이 있고, 후자는 세팅하기는 어렵지만 새 데이터를 업데이트하기 쉬우며 텍스트 또한 저장하기 쉽다는 장점이 있습니다.  
상황과 필요에 맞는 적절한 라이브러리를 활용해야겠습니다.